# A1_4 – Regularisation

In this notebook we compare different regularisation techniques applied to our neturalnet_torch model.

More specifically, we compare:

- L1/L2 Regularisation
- Dropout Regularisation

For each:

- We experiment with different parameters
- We present the results of the evaluation

In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os
import joblib

base = os.path.dirname(os.getcwd())  
sys.path.append(os.path.join(base, "models"))
sys.path.append(os.path.join(base, "utils"))

from sklearn.metrics import mean_squared_error, mean_absolute_error

from utils import predict_batch, mape, evaluate_regression
from NeuralNet import NeuralNet                      # manual BP implementation
from mlr_sklearn import MultipleLinearRegressionSK   # simple MLR wrapper
from neuralnet_torch import NeuralNetTorch           # PyTorch implementation

In [46]:
# Load preprocessed data from ./data
X_trainval_np = np.load("../data/X_trainval_np.npy")
X_test_np     = np.load("../data/X_test_np.npy")

y_trainval = np.load("../data/y_trainval.npy")
y_test     = np.load("../data/y_test.npy")

y_trainval_scaled = np.load("../data/y_trainval_scaled.npy")
y_test_scaled     = np.load("../data/y_test_scaled.npy")

x_scaler = joblib.load("../data/x_scaler.joblib")
y_scaler = joblib.load("../data/y_scaler.joblib")

n_features = X_trainval_np.shape[1]
print("Loaded preprocessed data from ../data")
print("X_trainval_np:", X_trainval_np.shape)
print("X_test_np    :", X_test_np.shape)
print("n_features   :", n_features)

Loaded preprocessed data from ../data
X_trainval_np: (1200, 61)
X_test_np    : (300, 61)
n_features   : 61


In [47]:
# Selected configuration for manual BP, we copy here the desired Hyperpaarameters from the notebook 2

hidden_layers_bp = [40, 15]
epochs_bp = 600
lr_bp = 0.005
momentum_bp = 0.9
activation_bp = "tanh"

print("Manual BP selected configuration:")
print("Hidden layers :", hidden_layers_bp)
print("Epochs        :", epochs_bp)
print("Learning rate :", lr_bp)
print("Momentum      :", momentum_bp)
print("Activation    :", activation_bp)


Manual BP selected configuration:
Hidden layers : [40, 15]
Epochs        : 600
Learning rate : 0.005
Momentum      : 0.9
Activation    : tanh


In [48]:
# PyTorch Neural Network 

hidden_layers_torch = hidden_layers_bp
layers_torch = [n_features] + hidden_layers_torch + [1]

results = []   
configs = [
    {"name": "None",    "reg_type": "None", "dropout": 0.0, "lambda_reg": 0.0},
    
    {"name": "L1",      "reg_type": "L1", "dropout": 0.0, "lambda_reg": 1e-1},
    {"name": "L1",      "reg_type": "L1", "dropout": 0.0, "lambda_reg": 1e-3},
    {"name": "L1",      "reg_type": "L1", "dropout": 0.0, "lambda_reg": 1e-5},

    {"name": "L2",      "reg_type": "L2", "dropout": 0.0, "lambda_reg": 1e-1},
    {"name": "L2",      "reg_type": "L2", "dropout": 0.0, "lambda_reg": 1e-3},
    {"name": "L2",      "reg_type": "L2", "dropout": 0.0, "lambda_reg": 1e-5},
    
    {"name": "dropout", "reg_type": "Dropout", "dropout": 0.2, "lambda_reg": 0.0},
    {"name": "dropout", "reg_type": "Dropout", "dropout": 0.3, "lambda_reg": 0.0},
    {"name": "dropout", "reg_type": "Dropout", "dropout": 0.4, "lambda_reg": 0.0},
]

for cfg in configs:    
    print(f"\n=== Training with {cfg['name']}, Lambda:{cfg['lambda_reg']}, Dropout:{cfg['dropout']} ===")

    net_torch = NeuralNetTorch(
    n=layers_torch,
    fact=activation_bp,   # activation
    eta=lr_bp,            # learning rate
    alpha=momentum_bp,    # momentum
    epochs=epochs_bp,     # number of epochs
    val_split=0.2,        # validation split,
    dropout=cfg['dropout']) # dropout
    
    # Train with scaled data
    net_torch.fit(X_trainval_np, y_trainval_scaled, cfg['reg_type'], cfg['lambda_reg'])

    # Loss history for later plots
    train_err_torch, val_err_torch = net_torch.loss_epochs()
    
    # Predictions in scaled space
    y_trainval_pred_torch_scaled = net_torch.predict(X_trainval_np).reshape(-1, 1)
    y_test_pred_torch_scaled     = net_torch.predict(X_test_np).reshape(-1, 1)
    
    # Back to original target scale (cnt_log)
    y_trainval_pred_torch = y_scaler.inverse_transform(y_trainval_pred_torch_scaled).ravel()
    y_test_pred_torch     = y_scaler.inverse_transform(y_test_pred_torch_scaled).ravel()
    
    # Metrics (in original cnt_log scale)
    metrics_trainval = evaluate_regression(y_trainval, y_trainval_pred_torch)
    metrics_test     = evaluate_regression(y_test,     y_test_pred_torch)

    print("=== PyTorch Neural Network (same config) ===")
    print("TRAIN+VAL:", metrics_trainval)
    print("TEST     :", metrics_test)

    # Store results
    # Add TRAIN+VAL row
    results.append({
        "Regularisation": cfg['reg_type'],
        "Split": "Train+Val",
        "MSE":  metrics_trainval["MSE"],
        "MAE":  metrics_trainval["MAE"],
        "MAPE": metrics_trainval["MAPE"],
        "Lambda": cfg['lambda_reg'],
        "Dropout": cfg['dropout']
    })

    # Add TEST row
    results.append({
        "Regularisation": cfg['reg_type'],
        "Split": "Test",
        "MSE":  metrics_test["MSE"],
        "MAE":  metrics_test["MAE"],
        "MAPE": metrics_test["MAPE"],
        "Lambda": cfg['lambda_reg'],
        "Dropout": cfg['dropout']
    })



=== Training with None, Lambda:0.0, Dropout:0.0 ===
NeuralNetTorch (PyTorch) initialized
 - Layers: [61, 40, 15, 1]
 - Activation: tanh
 - Learning rate: 0.005 | Momentum: 0.9
 - Epochs: 600 | Val split: 0.2
Epoch 0: Train MSE=1.084820 | Val MSE=1.044394
Epoch 100: Train MSE=0.093052 | Val MSE=0.088417
Epoch 200: Train MSE=0.077228 | Val MSE=0.077146
Epoch 300: Train MSE=0.064304 | Val MSE=0.067391
Epoch 400: Train MSE=0.052467 | Val MSE=0.058068
Epoch 500: Train MSE=0.042387 | Val MSE=0.049957
=== PyTorch Neural Network (same config) ===
TRAIN+VAL: {'MSE': 0.07214226063317045, 'MAE': 0.1888911920070034, 'MAPE': 6.68208138475534}
TEST     : {'MSE': 0.11174460046264317, 'MAE': 0.2046340380437264, 'MAPE': 8.83706912394687}

=== Training with L1, Lambda:0.1, Dropout:0.0 ===
NeuralNetTorch (PyTorch) initialized
 - Layers: [61, 40, 15, 1]
 - Activation: tanh
 - Learning rate: 0.005 | Momentum: 0.9
 - Epochs: 600 | Val split: 0.2
Epoch 0: Train MSE=21.943699 | Val MSE=1.141071
Epoch 100: Tr

In [51]:

# Comparison tables: TRAIN+VAL and TEST metrics
df_results = pd.DataFrame(results)

print("=== Evaluation metrics w/ Regularisation ===")
display(df_results)

print("=== Baseline -> no Regularisation ===")
baseline = df_results[df_results["Regularisation"] == "None"].set_index("Split")
display(baseline)

=== Evaluation metrics w/ Regularisation ===


,Regularisation,Split,MSE,MAE,MAPE,Lambda,Dropout
0,None,Train+Val,0.072142,0.188891,6.682081,0.00000,0.0
1,None,Test,0.111745,0.204634,8.837069,0.00000,0.0
2,L1,Train+Val,1.977083,1.156762,39.387823,0.10000,0.0
3,L1,Test,1.819883,1.099224,39.256915,0.10000,0.0
4,L1,Train+Val,0.104615,0.206674,7.855842,0.00100,0.0
5,L1,Test,0.141146,0.222801,9.655398,0.00100,0.0
6,L1,Train+Val,0.102890,0.211209,7.818223,0.00001,0.0
7,L1,Test,0.144142,0.229302,10.011982,0.00001,0.0
8,L2,Train+Val,0.221709,0.330829,12.444089,0.10000,0.0
9,L2,Test,0.246150,0.328380,14.175482,0.10000,0.0


=== Baseline -> no Regularisation ===


,Regularisation,MSE,MAE,MAPE,Lambda,Dropout
Split,,,,,,
Train+Val,None,0.072142,0.188891,6.682081,0.0,0.0
Test,None,0.111745,0.204634,8.837069,0.0,0.0


## Summary 
Regularisation did not improve the results. This is in part due to the small size of the dataset and, presumably, also because the dataset does not contain much noise. L2 perform slightl better than L1 and dropout, because it preserves useful information. 